Install libraries:

1) Install miniconda from the website
2) Add conda to PATH:
     - Start Menu and search for "Environment Variables."
     - Click on "Edit the system environment variables."
     - In the System Properties window, click on "Environment Variables."
     - In the Environment Variables window, find and select the Path.
     - Add new: C:\ProgramData\miniconda3\Scripts (or where yours is located).

3) Install libs:

Create a new virtual environment named 'langchain' with Python 3.12
```
conda create -n langchain python=3.12 -y
```
List all environments (the active one will be marked with an asterisk)
```
conda env list
```
In PowerShell (admin): Set the execution policy to RemoteSigned for the current user
```
Set-ExecutionPolicy RemoteSigned -Scope CurrentUser
```

```
conda activate langchain
pip install langchain langchain_openai langchain_community langgraph ipykernel python-dotenv
```

If you encounter a dependency resolver error with pip:
```
pip install google-auth # Must be <3.0.dev0,>=2.14.1
```

```
ipython kernel install --user --name=langchain
pip install google-search-results
pip install matplotlib
```

In [2]:
import os
import matplotlib.pyplot as plt
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_community.agent_toolkits.load_tools import get_all_tool_names

In [3]:
# load_tools() is only a shorthand function. It's better to properly use langchain_community.tools
all_tool_names = get_all_tool_names()
print('length: ', len(all_tool_names))

for i in range(0, len(all_tool_names), 10):
    print(" ".join(all_tool_names[i:i+10]))

length:  52
sleep wolfram-alpha google-search google-search-results-json searx-search-results-json bing-search metaphor-search ddg-search google-books google-lens
google-serper google-scholar google-finance google-trends google-jobs google-serper-results-json searchapi searchapi-results-json serpapi dalle-image-generator
twilio searx-search merriam-webster wikipedia arxiv golden-query pubmed human awslambda stackexchange
sceneXplain graphql openweathermap-api dataforseo-api-search dataforseo-api-search-json eleven_labs_text2speech google_cloud_texttospeech read_file reddit_search news-api
tmdb-api podcast-api memorize llm-math open-meteo-api requests requests_get requests_post requests_patch requests_put
requests_delete terminal


Types of LLMs:
- Language models - input stings and generate strings. They are typically older and work best to answer individual user queries.
- Chat model - inputs a sequence of messages, and generates responses that are contextually aware of the conversation flow. By default does not remember past conversations (stateless), but sometimes conversational memory is implemented. 
- Instruct models - optimized to follow specific instructions and perform tasks rather than engage in open-ended dialogue. 

In [4]:
# Chat model

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

chat_model = ChatOpenAI(api_key=api_key, model='gpt-4o-mini')


messages = [SystemMessage(content='You are a Scientist in AI.'),             # instruct a system to behave in certain way: tone, rules, how respond
            HumanMessage(content="Which is the most hot topic in AI now?")]  # input or queries made by the user/human    

# Test chat model 
chat_model.invoke(messages)

AIMessage(content="As of my last update in October 2023, some of the hottest topics in AI included:\n\n1. **Generative AI**: This encompasses advancements in models like GPT (including fine-tuning for specific applications), image generation models (like DALL-E), and multi-modal AI systems that can integrate text, image, and audio processing.\n\n2. **AI Ethics and Regulation**: With the rapid deployment of AI technologies, there is an increasing focus on ethical considerations, biases in AI systems, and the need for regulatory frameworks to ensure responsible use.\n\n3. **Explainable AI (XAI)**: As AI systems become more complex, understanding and explaining their decisions to users has gained prominence. Researchers aim to develop methods that make AI's decision-making processes more transparent.\n\n4. **AI Safety and Robustness**: Ensuring that AI systems are secure, reliable, and resistant to adversarial attacks is critical, particularly as they are deployed in more sensitive areas 

In [5]:
# Tools are classes that an Agent uses to interact with the world. https://python.langchain.com/api_reference/community/tools.html 

from langchain_community.tools.google_jobs import GoogleJobsQueryRun
from langchain_community.utilities.google_jobs import GoogleJobsAPIWrapper

api_key_serpapi = os.getenv('SERPAPI_API_KEY')
os.environ["SERPAPI_API_KEY"] = api_key_serpapi

# Take special care of description
description_googlejobs = """
A wrapper around Google Jobs Search. 
Useful for when you need to get information aboutgoogle search Jobs from Google JobsInput should be a search query. 
"""

googlejobs = GoogleJobsQueryRun(api_wrapper=GoogleJobsAPIWrapper(), 
                                     description=description_googlejobs)

# Example usage or run(): 
googlejobs.run("Can I get a list of 3 job posting related to machine learning in alberta")

"\n_______________________________________________\nJob Title: Machine Learning Resident - Client: Jotson (12 months)\nCompany Name: Alberta Machine Intelligence Institute\nLocation: Edmonton, AB, Canada\nDescription: Salary:\n\nJoin us for a unique ML Resident role tackling time-series problems in the energy domain with ML/DL. Youll collaborate with a dynamic and fast-paced team of machine learning scientists and domain experts, developing innovative models and products with energy data.\n\n- Maithrreye Srinivasan, Machine Learning Scientist and Dave Staszak, Lead Machine Learning Scientist\n\nAbout the Role\n\nThis is a paid residency that will be undertaken over a twelve-month period with the potential to be hired by our client afterwards (note: at the discretion of the client and with the requirement of being located in Calgary at that time). The resident will be reporting to an Amii Machine Learning Scientist and regularly consult with the Client team to share insights and engage 

In [6]:
tools = [googlejobs,]

- This agent from LangChain is not good - langchain.agents.react.agent.create_react_agent
- Better use from the LangGraph (offers a more flexible and full-featured)

There are agent types in LangChain:

- React agents: Use reasoning and actions to decide on the best steps 
- Multi-agent: Divide complicated problems into units of work that can be targeted by specialized agents 
- Conversational agents: Engage in dialogue and maintain context across multiple interactions 
- Structured chat agents: Parse inputs and outputs into structured formats 
- Tool calling agents: Use tools in a straightforward way 
- Self-ask with search: Split queries into smaller steps to handle them 

In [17]:
from langgraph.prebuilt import create_react_agent
 
prompt_system = SystemMessage("You are a recruiter, who is trying to help me in finding suitable jobs")
prompt_human  = HumanMessage("Find me the most recent Machine Learning job in Alberta, specify when the job was posted")

agent = create_react_agent(chat_model,
                           tools,
                           state_modifier=prompt_system)

response = agent.invoke({"messages": prompt_human})

In [18]:
print(response['messages'][3].content)

The most recent Machine Learning job in Alberta is for the position of **Machine Learning Engineer** at **Ontopical** in Calgary, AB, Canada. 

### Job Details:
- **Posted Date:** Not specified in the job details provided
- **Location:** Calgary, AB or Vancouver, BC
- **Description:** The role involves developing, deploying, and maintaining production-ready scalable ML models, along with other responsibilities related to NLP research and business insights.

If you would like to know more about the application process or require additional information, feel free to ask!
